In [131]:
import warnings
from typing import Literal

import network_diffusion as nd
from misc.utils import extract_basic_stats
from tqdm import tqdm


warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [137]:
def get_sigma(
    S_0: set[nd.MLNetworkActor],
    A: set[nd.MLNetworkActor],
    M: nd.MultilayerNetwork,
    mi: float,
    delta: Literal["OR", "AND"],
) -> int:
    """Obtain sigma for given network, seed set and MLTM params."""
    A_order = [*S_0, *A.difference(S_0)]
    S_0_card = 100 * len(S_0) / len(A)
    model = nd.models.MLTModel(
        seeding_budget=[100 - S_0_card, S_0_card],
        seed_selector=nd.seeding.MockyActorSelector(A_order),
        protocol=delta,
        mi_value=mi,
    )
    logger = nd.Simulator(model, M).perform_propagation(10000, 1)
    return extract_basic_stats(logger._local_stats)[1]

In [6]:
net = nd.mln.functions.get_toy_network_piotr()
actors = net.get_actors()
mi_vals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [149]:
slen = 0
i = 0

delta = "AND"
mi = 0.5

for S in powerset(actors):
    sigma_S = get_sigma(S, actors, net, mi, delta)
    ids_S = [a.actor_id for a in S]

    for a in set(actors).difference(S):
        sigma_S_a = get_sigma([*S, a], actors, net, mi, delta)
        id_a = a.actor_id

        for S_prime in powerset(S):
            if len(S_prime) == 0:
                continue
            sigma_S_prime = get_sigma(S_prime, actors, net, mi, delta)
            sigma_S_prime_a = get_sigma([*S_prime, a], actors, net, mi, delta)     
            ids_S_prime = [a.actor_id for a in S_prime]

            strS = f"S: {ids_S}, a: {id_a}, S': {ids_S_prime}"
            strs = f"s(S' u {{a}}): {sigma_S_prime_a}, s(S'): {sigma_S_prime} ... s(S u {{a}}): {sigma_S_a}, s(S): {sigma_S}"

            assert sigma_S_prime <= sigma_S, f"{strS}   {strs}"
            assert sigma_S_prime_a - sigma_S_prime >= sigma_S_a - sigma_S, f"{strS}   {strs}"

AssertionError: S: [6, 5], a: 11, S': [6]   s(S' u {a}): 2, s(S'): 1 ... s(S u {a}): 4, s(S): 2

In [147]:
# AssertionError: S: [6], a: 5, S': []   s(S'u{a}): 1, s(S'): 0 ... s(Su{a}): 11, s(S): 1
# AssertionError: S: [6, 11], a: 5, S': [11]   s(S' u {a}): 3, s(S'): 2 ... s(S u {a}): 11, s(S): 3

(1, [MLNetworkActor: 5 at 4899807120])